In [1]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gbbxxhug
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gbbxxhug
  Resolved https://github.com/huggingface/transformers to commit c681b58b06f6fb8b5c331f380548af3b4b33f881
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of optimum to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 70.6 MB/s eta 0:

In [2]:
!nvidia-smi

Thu May  2 06:45:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  n


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'akshat1311/Mistral-DronaHQ-Control'

my_prompt = """Below is an instruction that describes a task,You must generate the full stringified json response without skipping anything.

### input:
{}

### output:
{}"""


def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto",
        load_in_4bit=True,# For 8 bit quantization,
    )


    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]


adapter_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:163: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(
[
    my_prompt.format(
        "Generate a splitbutton for acceptance status", # input
        "", # output - left blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
inputs = tokenizer(
[
    my_prompt.format(
        "Generate a button with text 'Akshat'", # input
        "", # output - left blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
inputs = tokenizer(
[
    my_prompt.format(
        "Generate a button3 with text 'SampleButton'", # input
        "", # output - left blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
tokenizer.batch_decode(outputs)

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [2]:
import evaluate
bleu = evaluate.load("bleu")

2024-05-02 10:20:15.366238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 10:20:15.366567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 10:20:15.571804: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
predictions = ["{\\\"ctrl_id\\\":\\\"\\\",\\\"label\\\":\\\"Split Button\\\",\\\"label_font_weight_class\\\":\\\"font-weight-normal\\\",\\\"label_position\\\":\\\"top\\\",\\\"label_left_width\\\":\\\"33\\\",\\\"label_left_width_unit\\\":\\\"%\\\",\\\"label_font_size\\\":\\\"var(--DHQ-control-title)\\\",\\\"label_font_size_unit\\\":\\\"\\\",\\\"label_font_weight\\\":\\\"var(--DHQ-bold)\\\",\\\"label_text_alignment\\\":\\\"left\\\",\\\"label_text_color\\\":\\\"var(--DHQ-primary-font-color)\\\",\\\"label_hidden\\\":true,\\\"undefined\\\":\\\"flex-start\\\",\\\"field_type\\\":\\\"dhq_split_button\\\",\\\"field_key_name\\\":\\\"dhq_split_button_c1638R1JJRlxJQ\\\",\\\"field_display_key_name\\\":\\\"splitbutton\\\",\\\"field_responsive\\\":false,\\\"caching\\\":false,\\\"grid_dynamic_height\\\":true,\\\"grid_maintain_space\\\":false,\\\"grid_max_height\\\":\\\"\\\",\\\"grid_max_height_unit\\\":\\\"px\\\",\\\"type\\\":\\\"non_input\\\",\\\"required\\\":false,\\\"hidden\\\":false,\\\"disabled\\\":false,\\\"data_submission_enabled\\\":true,\\\"is_variable\\\":false,\\\"tooltip\\\":\\\"\\\",\\\"tip_description\\\":\\\"\\\",\\\"show_description\\\":false,\\\"show_tooltip\\\":false,\\\"field_fullview\\\":false,\\\"class_name\\\":\\\"test\\\",\\\"field_data_model\\\":\\\"\\\",\\\"allow_options_addition_in_editor\\\":true,\\\"field_composition_type\\\":\\\"single\\\",\\\"field_editable_in_database_editor\\\":true,\\\"field_values_addition_enabled_in_database_editor\\\":true,\\\"field_values_source\\\":\\\"\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_data_model_binding_manual_query\\\":\\\"DATABUILDER()\\\",\\\"field_data_model_binding_manual_query_with_id\\\":\\\"DATABUILDER()\\\",\\\"show_dfm\\\":false,\\\"dfmv_selected\\\":\\\"default\\\",\\\"dhq_formula\\\":\\\"[{\\\"value\\\":\\\"Accept\\\",\\\"label\\\":\\\"Accept\\\"},{\\\"value\\\":\\\"Reject\\\",\\\"label\\\":\\\"Reject\\\"},{\\\"value\\\":\\\"Neutral\\\",\\\"label\\\":\\\"Neutral\\\"}]\\\",\\\"dhq_data_process\\\":\\\"\\\",\\\"dhq_default_data_process\\\":\\\"\\\",\\\"dhq_default_formula\\\":\\\"\\\",\\\"field_data_model_binding_formula\\\":\\\"\\\",\\\"field_data_model_binding_type\\\":\\\"manual_query\\\",\\\"validations\\\":[],\\\"formbuilder_theme\\\":\\\"var(--DHQ-border-color)\\\",\\\"field_options\\\":{},\\\"field_config\\\":{},\\\"input_placeholder_text_size\\\":\\\"var(--DHQ-small-font)\\\",\\\"input_placeholder_text_color\\\":\\\"#aaaaaa\\\",\\\"input_placeholder_font_weight\\\":\\\"var(--DHQ-normal-weight)\\\",\\\"input_control_height\\\":\\\"2.5rem\\\",\\\"input_font_color\\\":\\\"var(--DHQ-primary-font-color)\\\",\\\"input_font_weight\\\":\\\"var(--DHQ-normal-weight)\\\",\\\"input_font_size\\\":\\\"var(--DHQ-normal-font)\\\",\\\"is_master_control\\\":false,\\\"is_outlet_control\\\":false,\\\"is_repeat_outlet_control\\\":false,\\\"dhq_req_stats\\\":{\"status\\\":\\\"\\\"},\\\"dhq_nsf_replaced\\\":false,\\\"dhq_olddt_map\\\":{},\\\"dhq_submit_condition\\\":\\\"always\\\",\\\"dhq_tfd\\\":true,\\\"dhq_external_css\\\":{\"addCode\\\":false,\\\"data\\\":{}},\\\"tabIndex\\\":\\\"\\\",\\\"statesObj\\\":{},\\\"field_available_in_database_editor\\\":false,\\\"field_client_side_get_data_enabled\\\":true,\\\"field_data_model_binding_formula_with_id\\\":\\\"\\\",\\\"field_data_model_binding_manual_query_with_id\\\":\\\"DATABUILDER()\\\",\\\"field_data_model_binding_type_with_id\\\":\\\"manual_query\\\",\\\"dhq_config\\\":{\"strict_update\\\":[\\\"dhq_config\\\"],\\\"update_to_theme_varaibles\\\":[\\\"formbuilder_theme\\\"],\\\"theme_variable_mapping\\\":{\"formbuilder_theme\\\":\\\"DHQ_CTRL_THEME\\\"}},\\\"event_items\\\":{\"button_click\\\":{\"click\\\":[{\\\"formula_with_id\\\":\\\"\\\",\\\"formula_with_name\\\":\\\"\\\"}]}},\\\"dhq_keywords_mapping\\\":[],\\\"dhq_config_change_events\\\":[\\\"dhq_config\\\"],\\\"field_datatypes\\\":[],\\\"field_data_models\\\":[],\\\"sb_mode\\\":\\\"JS\\\",\\\"LiveData\\\":\\\"[{\\\"value\\\":\\\"Accept\\\",\\\"label\\\":\\\"Accept\\\"},{\\\"value\\\":\\\"Reject\\\",\\\"label\\\":\\\"Reject\\\"},{\\\"value\\\":\\\"Neutral\\\",\\\"label\\\":\\\"Neutral\\\"}]\\\",\\\"dhqGrid\\\":{\"w\\\":6,\\\"h\\\":6,\\\"x\\\":1,\\\"y\\\":1},\\\"dhqGridDesktop\\\":{\"w\\\":6,\\\"h\\\":6,\\\"x\\\":1,\\\"y\\\":1},\\\"isDhqGridCtrl\\\":true,\\\"cid\\\":\\\"c1638R1JJRlxJQ\\\",\\\"sid\\\":\\\"sc1638R1JJRlxJQ\\\",\\\"actionflows\\\":{\"button_click\\\":{\"click\\\":{\"event_name\\\":\\\"button_click\\\",\\\"selector\\\":\\\"[eventname='button_click' and eventtype='click']\\\",\\\"event_type\\\":\\\"click\\\",\\\"actionflow_id\\\":\\\"af_dhq_split_button_c1638R1JJRlxJQ_button_click_click\\\",\\\"next_actions\\\":[\\\"end_action_flow\\\"],\\\"prev_actions\\\":[],\\\"actions\\\":[],\\\"category\\\":\\\"\\\",\\\"interval\\\":\\\"\\\",\\\"action_label\\\":\\\"\\\"}}},\\\"dhqEvtConverted\\\":true,\\\"dhq_keywords_mapping_converted\\\":[],\\\"sb_submit_condition\\\":\\\"always\\\",\\\"dhqMGridChanged\\\":true,\\\"dhqMGridChangedDesktop\\\":true,\\\"dhqMGridChangedMobile\\\":true,\\\"isDhqMGridCtrl\\\":true,\\\"LiveDataWithId\\\":\\\"[{\\\"value\\\":\\\"Accept\\\",\\\"label\\\":\\\"Accept\\\"},{\\\"value\\\":\\\"Reject\\\",\\\"label\\\":\\\"Reject\\\"},{\\\"value\\\":\\\"Neutal\\\",\\\"label\\\":\\\"Neutal\\\"}]\\\",\\\"dhqGridMobile\\\":{\"w\\\":6,\\\"h\\\":6,\\\"x\\\":1,\\\"y\\\":1},\\\"dhqGridDesktop\\\":{\"w\\\":6,\\\"h\\\":6,\\\"x\\\":1,\\\"y\\\":1},\\\"dhqGridOld\\\":{\"w\\\":6,\\\"h\\\":6,\\\"x\\\":1,\\\"y\\\":1},\\\"controls\\\":[]}"]
references = ["{\\\"ctrl_id\\\":\\\"\\\",\\\"label\\\":\\\"Split Button\\\",\\\"label_font_weight_class\\\":\\\"font-weight-normal\\\",\\\"label_position\\\":\\\"top\\\",\\\"label_left_width\\\":\\\"33\\\",\\\"label_left_width_unit\\\":\\\"%\\\",\\\"label_font_size\\\":\\\"var(--DHQ-control-title)\\\",\\\"label_font_size_unit\\\":\\\"\\\",\\\"label_font_weight\\\":\\\"var(--DHQ-bold)\\\",\\\"label_text_alignment\\\":\\\"left\\\",\\\"label_text_color\\\":\\\"var(--DHQ-primary-font-color)\\\",\\\"label_hidden\\\":false,\\\"undefined\\\":\\\"flex-start\\\",\\\"field_type\\\":\\\"dhq_split_button\\\",\\\"field_key_name\\\":\\\"dhq_split_button_c26152Rq5536PxIO539fpuA\\\",\\\"field_display_key_name\\\":\\\"decision\\\",\\\"field_responsive\\\":false,\\\"caching\\\":false,\\\"grid_dynamic_height\\\":true,\\\"grid_maintain_space\\\":false,\\\"grid_max_height\\\":\\\"\\\",\\\"grid_max_height_unit\\\":\\\"px\\\",\\\"type\\\":\\\"non_input\\\",\\\"required\\\":false,\\\"hidden\\\":false,\\\"disabled\\\":false,\\\"data_submission_enabled\\\":true,\\\"is_variable\\\":false,\\\"tooltip\\\":\\\"\\\",\\\"tip_description\\\":\\\"\\\",\\\"show_description\\\":false,\\\"show_tooltip\\\":false,\\\"field_fullview\\\":false,\\\"class_name\\\":\\\"test\\\",\\\"field_data_model\\\":\\\"\\\",\\\"allow_options_addition_in_editor\\\":true,\\\"field_composition_type\\\":\\\"list\\\",\\\"field_editable_in_database_editor\\\":false,\\\"field_values_addition_enabled_in_database_editor\\\":true,\\\"field_values_source\\\":\\\"\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_data_model_binding_manual_query\\\":\\\"DATABUILDER()\\\",\\\"field_data_model_binding_manual_query_with_id\\\":\\\"DATABUILDER()\\\",\\\"field_data_model_binding_default_value_query\\\":\\\"\\\",\\\"field_data_model_binding_default_value_query_with_id\\\":\\\"\\\",\\\"show_dfm\\\":false,\\\"dfmv_selected\\\":\\\"default\\\",\\\"field_schema_column_header_names\\\":\\\"value,label,description,image\\\",\\\"field_schema_column_id_header_name_mapping\\\":{\\\"value\\\":\\\"value\\\",\\\"label\\\":\\\"label\\\",\\\"description\\\":\\\"description\\\",\\\"image\\\":\\\"image\\\"},\\\"dhq_formula\\\":\\\"[{\\\\\\\"value\\\\\\\":\\\\\\\"accept\\\\\\\",\\\\\\\"label\\\\\\\":\\\\\\\"Accept\\\\\\\",\\\\\\\"description\\\\\\\":\\\\\\\"John@studio.com\\\\\\\",\\\\\\\"image\\\\\\\":\\\\\\\"https://dronamobilepublic.s3.amazonaws.com/DRONA5_Team108/content/app/images/public/Tick_c5Abe.svg\\\\\\\"},{\\\\\\\"value\\\\\\\":\\\\\\\"reject\\\\\\\",\\\\\\\"label\\\\\\\":\\\\\\\"Reject\\\\\\\",\\\\\\\"description\\\\\\\":\\\\\\\"Dave@studio.com\\\\\\\",\\\\\\\"image\\\\\\\":\\\\\\\"https://dronamobilepublic.s3.amazonaws.com/DRONA5_Team108/content/app/images/public/Close_L8P9q.svg\\\\\\\"}]\\\",\\\"dhq_data_process\\\":\\\"\\\",\\\"dhq_default_data_process\\\":\\\"\\\",\\\"dhq_default_formula\\\":\\\"\\\",\\\"field_data_model_binding_formula\\\":\\\"\\\",\\\"field_data_model_binding_type\\\":\\\"manual_query\\\",\\\"validations\\\":[],\\\"formbuilder_theme\\\":\\\"#bbbbbb\\\",\\\"field_options\\\":{\\\"main_config\\\":{\\\"gridMobileMaxHeight\\\":880}},\\\"field_config\\\":{},\\\"input_placeholder_text_size\\\":\\\"var(--DHQ-small-font)\\\",\\\"input_placeholder_text_color\\\":\\\"#aaaaaa\\\",\\\"input_placeholder_font_weight\\\":\\\"var(--DHQ-normal-weight)\\\",\\\"input_control_height\\\":\\\"2.5rem\\\",\\\"input_font_color\\\":\\\"var(--DHQ-primary-font-color)\\\",\\\"input_font_weight\\\":\\\"var(--DHQ-normal-weight)\\\",\\\"input_font_size\\\":\\\"var(--DHQ-normal-font)\\\",\\\"is_master_control\\\":true,\\\"is_outlet_control\\\":false,\\\"is_repeat_outlet_control\\\":false,\\\"dhq_req_stats\\\":{\\\"status\\\":\\\"\\\"},\\\"dhq_nsf_replaced\\\":false,\\\"dhq_olddt_map\\\":{\\\"value\\\":\\\"value\\\",\\\"label\\\":\\\"label\\\",\\\"description\\\":\\\"description\\\",\\\"image\\\":\\\"image\\\"},\\\"dhq_submit_condition\\\":\\\"always\\\",\\\"dhq_tfd\\\":true,\\\"dhq_external_css\\\":{\\\"addCode\\\":false,\\\"data\\\":{}},\\\"tabIndex\\\":\\\"\\\",\\\"title_font_size\\\":\\\"var(--DHQ-medium-font)\\\",\\\"title_font_size_unit\\\":\\\"\\\",\\\"title_text_color\\\":\\\"#000000\\\",\\\"title_font_weight\\\":\\\"var(--DHQ-bold)\\\",\\\"dropdown_title_font_size\\\":\\\"var(--DHQ-normal-font)\\\",\\\"dropdown_font_size_unit\\\":\\\"\\\",\\\"dropdown_title_font_weight\\\":\\\"var(--DHQ-bold)\\\",\\\"dropdown_title_text_color\\\":\\\"#000000\\\",\\\"dropdown_desc_font_size\\\":\\\"var(--DHQ-x-small-font)\\\",\\\"dropdown_desc_font_size_unit\\\":\\\"\\\",\\\"dropdown_desc_font_weight\\\":\\\"var(--DHQ-normal-weight)\\\",\\\"dropdown_desc_font_color\\\":\\\"#000000\\\",\\\"dropdown_card_background_color\\\":\\\"#ffdd02\\\",\\\"dropdown_card_hover_color\\\":\\\"#bababa\\\",\\\"card_background_color\\\":\\\"#ffdd00\\\",\\\"btn_label\\\":\\\"--\\\",\\\"card_theme\\\":\\\"filled-theme\\\",\\\"side_to_display\\\":\\\"right\\\",\\\"card_border_radius\\\":\\\"var(--DHQ-border-radius)\\\",\\\"card_border_radius_unit\\\":\\\"\\\",\\\"title_column\\\":\\\"label\\\",\\\"desc_column\\\":\\\"description\\\",\\\"value_column\\\":\\\"value\\\",\\\"image_column\\\":\\\"image\\\",\\\"image_radius\\\":3.125,\\\"image_radius_unit\\\":\\\"rem\\\",\\\"label_value\\\":\\\"Sample Label\\\",\\\"field_available_in_database_editor\\\":false,\\\"field_client_side_get_data_enabled\\\":true,\\\"hide_list_dfm\\\":true,\\\"action_items\\\":{\\\"button_click\\\":{\\\"events\\\":[\\\"click\\\"],\\\"selector\\\":\\\".control-button\\\"}},\\\"dhq_keywords_mapping\\\":[],\\\"dhq_config\\\":{\\\"strict_update\\\":[\\\"for_localization\\\",\\\"ui_input_config\\\",\\\"dhq_config\\\"],\\\"update_to_theme_varaibles\\\":[\\\"title_font_size\\\",\\\"image_radius\\\",\\\"image_radius_unit\\\",\\\"card_border_radius\\\",\\\"card_border_radius_unit\\\",\\\"title_font_size_unit\\\",\\\"title_text_color\\\",\\\"title_font_weight\\\",\\\"dropdown_title_font_size\\\",\\\"dropdown_font_size_unit\\\",\\\"dropdown_title_font_weight\\\",\\\"dropdown_title_text_color\\\",\\\"dropdown_desc_font_size\\\",\\\"dropdown_desc_font_size_unit\\\",\\\"dropdown_desc_font_weight\\\",\\\"dropdown_desc_font_color\\\",\\\"dropdown_card_background_color\\\",\\\"dropdown_card_hover_color\\\",\\\"card_background_color\\\",\\\"label_font_weight\\\",\\\"label_font_size\\\",\\\"label_font_size_unit\\\",\\\"label_font_weight\\\",\\\"label_text_color\\\"]},\\\"enable_ui_input\\\":true,\\\"sb_mode\\\":{\\\"main\\\":\\\"UI\\\"},\\\"ui_input_config\\\":{\\\"type\\\":\\\"JSON_BUILDER\\\",\\\"label\\\":\\\"Menu Options\\\",\\\"option\\\":{\\\"schema\\\":{\\\"label\\\":\\\"string\\\",\\\"value\\\":\\\"string\\\",\\\"image\\\":\\\"file_upload\\\",\\\"description\\\":\\\"string\\\"},\\\"default\\\":{\\\"label\\\":\\\"Option\\\",\\\"value\\\":\\\"\\\",\\\"image\\\":\\\"\\\",\\\"description\\\":\\\"\\\"},\\\"option_label_key\\\":\\\"label\\\",\\\"statusValues\\\":[{\\\"Label1\\\":\\\"value1\\\"},{\\\"Label2\\\":\\\"value2\\\"},{\\\"Label3\\\":\\\"value3\\\"},{\\\"Label4\\\":\\\"value4\\\"}],\\\"enable_children\\\":false,\\\"child_levels\\\":2,\\\"children_key\\\":\\\"children\\\"}},\\\"field_datatypes\\\":[{\\\"field\\\":\\\"value\\\",\\\"display_name\\\":\\\"value\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\"},{\\\"field\\\":\\\"label\\\",\\\"display_name\\\":\\\"label\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\"},{\\\"field\\\":\\\"description\\\",\\\"display_name\\\":\\\"description\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\"},{\\\"field\\\":\\\"image\\\",\\\"display_name\\\":\\\"image\\\",\\\"data_type\\\":\\\"string\\\",\\\"field_database_editor_data_format\\\":\\\"single_line_text\\\"}],\\\"for_localization\\\":{\\\"static_msgs\\\":[\\\"Please select title column and value column\\\"],\\\"dynamic_msgs_keys\\\":[\\\"label\\\",\\\"tooltip\\\",\\\"tip_description\\\"],\\\"concated_msgs\\\":[]},\\\"dhqGrid\\\":{\\\"w\\\":10,\\\"h\\\":8,\\\"x\\\":1,\\\"y\\\":74},\\\"dhqGridDesktop\\\":{\\\"w\\\":9,\\\"h\\\":8,\\\"x\\\":1,\\\"y\\\":67},\\\"dhqGridChanged\\\":false,\\\"isDhqGridCtrl\\\":true,\\\"isDhqHideOnDesktop\\\":false,\\\"isDhqHideOnMobile\\\":false,\\\"cid\\\":\\\"c26152Rq5536PxIO539fpuA\\\",\\\"sid\\\":\\\"sc26152Rq5536PxIO539fpuA\\\",\\\"actionflows\\\":{},\\\"dhqEvtConverted\\\":true,\\\"gridMobileOrigin\\\":null,\\\"dhqMGridChanged\\\":true,\\\"gridDesktopOrigin\\\":null,\\\"LiveData\\\":[{\\\"value\\\":\\\"accept\\\",\\\"label\\\":\\\"Accept\\\",\\\"description\\\":\\\"John@studio.com\\\",\\\"image\\\":\\\"https://dronamobilepublic.s3.amazonaws.com/DRONA5_Team108/content/app/images/public/Tick_c5Abe.svg\\\"},{\\\"value\\\":\\\"reject\\\",\\\"label\\\":\\\"Reject\\\",\\\"description\\\":\\\"Dave@studio.com\\\",\\\"image\\\":\\\"https://dronamobilepublic.s3.amazonaws.com/DRONA5_Team108/content/app/images/public/Close_L8P9q.svg\\\"}],\\\"controls\\\":[]}"]
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'bleu': 0.4682842318411662, 'precisions': [0.9368131868131868, 0.8662391204764086, 0.8111824014665444, 0.7491976157725814], 'brevity_penalty': 0.5588022042689691, 'length_ratio': 0.6321273516642547, 'translation_length': 2184, 'reference_length': 3455}


In [ ]:
#Example to understand the bleu evaluation.
#Bleu score is the overall score
#precision is based on [unigram,bigram,trigram,fourgram overlapping]
#brevity penality is if the predicted text length is shorter than the reference text.
#length ratio is the ratio of reference text lenth to the generated text length
#The reference here means the dataset input, and prediction means the model generated input
#translation length is predicted text length and reference length is reference text length

predictions = ["Hello, how are you"]
references = ["Hello, how are you"]
results = bleu.compute(predictions=predictions, references=references)
print(results)